In [149]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import folium
import requests
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors

# Load data

I load the data generated in the previous notebook and I verify the amount of boroughs and neigborhoods in the data

In [115]:
TorontoData = pd.read_hdf("TorontoData.hdf")
TorontoData.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [116]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoData['Borough'].unique()),
        TorontoData.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


# Let us explore the data collected in a map

I just use geolocator to get the geographical coordinates of Toronto to plot the information using Folium

In [117]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/usr/local/miniconda3/envs/sml/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [147]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = TorontoData

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Configure Foursquare access

In [175]:
CLIENT_ID = 'XXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXXXXXXX
CLIENT_SECRET:XXXXXX


I make use of a function that takes the names and locations of the neighborhoods in Toronto and obtains the 100 top venues around.

In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
Toronto_venues = getNearbyVenues(TorontoData.Neighbourhood,
                            TorontoData.Latitude,
                            TorontoData.Longitude)

100%|██████████| 103/103 [00:51<00:00,  2.37it/s]


I print the shape of my data and also a sample of the initial rows of information

In [122]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2255, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


I can check the amount of venues per neighboorhood

In [123]:
Toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
Harbourfront East, Toronto Islands, Union Station    100
St. James Town                                       100
Chinatown, Grange Park, Kensington Market            100
Commerce Court, Victoria Hotel                       100
Design Exchange, Toronto Dominion Centre             100
Name: Venue, dtype: int64

And the total amount of unique categories in my data

In [124]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


Let us now take the venue category information and create a dataframe with a one hot enconding of these data.

In [125]:
Toronto_OHE = pd.get_dummies(Toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_OHE["Neighborhood"] = Toronto_venues["Neighborhood"]


nindex = list(Toronto_OHE.columns).index("Neighborhood")
cols = deque(Toronto_OHE.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_OHE = Toronto_OHE[cols]

Toronto_OHE.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,...,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


As there are 276 unique categories, the datagrame contains all the neighborhoods and 271 columns

In [126]:
Toronto_OHE.shape

(2255, 276)

I can compute the average number of venue categories per neighborhood

In [91]:
Toronto_grouped = Toronto_OHE.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,...,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue
0,"Adelaide, King, Richmond",0.01,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In total I have 99 Neighborhoods, grouped by borough, and 271 distinct categories

In [101]:
Toronto_grouped.shape

(99, 271)

The number of Neighborhoods obtained from the venues is smaller than the original number of postcodes, what means that in some postcodes there are no venues

For neighborhood clustering let us select only the N most frequent venue categories per neighborhood

In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [104]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Gym,Restaurant,Bar,Hotel,Sushi Restaurant
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Music Venue,Antique Shop,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Music Venue,Arepa Restaurant,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Video Store,Fast Food Restaurant,Pizza Place,Pharmacy,Liquor Store,Beer Store,Fried Chicken Joint,Sandwich Place
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pub,Sandwich Place,Bank,Gym,Antique Shop,Airport Service,Airport Terminal


# Neighborhood clustering

Before clustering, let us use a PCA decomposition to reduce the noise of the signal and improve clustering efficiency

In [163]:
pca = PCA(.95)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

The dimensionality reduction has kept only a reduced number of the total dimensions

In [164]:
Toronto_grouped_clustering.shape

(99, 271)

With reduced dimensional dataset we carry out the KMeans clustering

In [165]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[0 0 2 4 0 0 0 0 0 0]
(99,)


Let us now create a dataframe that containes the neighborhood, the location and the cluster information, together with the top 10 venues

In [166]:
Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = TorontoData.merge(Toronto_grouped, left_on = "Neighbourhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",43.806686,-79.194353,"Rouge, Malvern",0.0,0.0,0.0,0.0,0.0,...,Fast Food Restaurant,Music Venue,Airport,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Arepa Restaurant,Aquarium
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.0,0.0,0.0,...,Construction & Landscaping,Bar,Aquarium,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Music Venue
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,...,Mexican Restaurant,Pizza Place,Medical Center,Electronics Store,Breakfast Spot,Rental Car Location,Music Venue,Airport Lounge,Airport Service,Airport Terminal
3,Scarborough,Woburn,43.770992,-79.216917,Woburn,0.0,0.0,0.0,0.0,0.0,...,Coffee Shop,Indian Restaurant,Korean Restaurant,Music Venue,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
4,Scarborough,Cedarbrae,43.773136,-79.239476,Cedarbrae,0.0,0.0,0.0,0.0,0.0,...,Caribbean Restaurant,Hakka Restaurant,Bank,Bakery,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Organic Grocery,Trail,Auto Workshop


In [167]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analyze clusters

## Cluster 1

Cluster 1 contains many restaurants and diners, as well as some other services, like bus and train stations

In [170]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 0, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Construction & Landscaping,Bar,Aquarium,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Music Venue
2,Mexican Restaurant,Pizza Place,Medical Center,Electronics Store,Breakfast Spot,Rental Car Location,Music Venue,Airport Lounge,Airport Service,Airport Terminal
3,Coffee Shop,Indian Restaurant,Korean Restaurant,Music Venue,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
4,Caribbean Restaurant,Hakka Restaurant,Bank,Bakery,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Organic Grocery,Trail,Auto Workshop
6,Discount Store,Bus Station,Chinese Restaurant,Train Station,Hobby Shop,Department Store,Coffee Shop,Antique Shop,Airport Lounge,Airport Service


## Cluster 2

Cluster 2 seems to be dominated by banks, music venues and airport facilities

In [171]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Bank,Park,Music Venue,Arepa Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium
94,Bank,Music Venue,Arepa Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery


## Cluster 3

Cluster 3 is mostly linked to open areas for sport and children

In [172]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Health & Beauty Service,Playground,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Arepa Restaurant,Afghan Restaurant,Antique Shop
14,Playground,Park,Music Venue,Arepa Restaurant,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
30,Playground,Bus Stop,Park,Airport,Music Venue,Aquarium,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
40,Convenience Store,Park,Music Venue,Aquarium,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant
50,Park,Trail,Playground,Music Venue,Arepa Restaurant,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium


## Cluster 4

Cluster 4 is the most homogeneous one (it only contains two neighborhoods) and seems to be dominated by the Baseball field

In [173]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Baseball Field,Music Venue,Arepa Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
97,Baseball Field,Music Venue,Arepa Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery


## Cluster 5

Cluster 5 is characterized by shops and restaurants

In [174]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 4, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Fast Food Restaurant,Music Venue,Airport,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery,Arepa Restaurant,Aquarium
24,Coffee Shop,Butcher,Pizza Place,Grocery Store,Pharmacy,Antique Shop,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
31,Grocery Store,Shopping Mall,Bank,Music Venue,Aquarium,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
33,Discount Store,Grocery Store,Athletics & Sports,Liquor Store,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
81,Convenience Store,Pizza Place,Grocery Store,Music Venue,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
